In [ ]:
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 60)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 100)

In [ ]:
import pandas as pd

wildfire_df = pd.read_csv("./../data/raw/california-wildfires/b8aeb030-140d-43d2-aa29-1a80862e3d62.csv", low_memory=False)

wildfire_df

In [ ]:
aqi_df = pd.read_csv("./../data/raw/us-air-quality/US_AQI.csv", low_memory=False)

neighboring_states = ["CA", "OR", "NV", "AZ"]  # State codes
aqi_neighbors_df = aqi_df[aqi_df["state_id"].isin(neighboring_states)]

aqi_neighbors_df

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

def plot_missing_data_heatmap(df):

    # Create a more informative heatmap
    plt.figure(figsize=(14, 8))

    # Calculate missing data percentage
    missing_percentage = (df.isnull().mean() * 100).sort_values(ascending=False)

    # Create a dataframe for missing data
    missing_df = pd.DataFrame({'Column': missing_percentage.index,
                              'Missing %': missing_percentage.values})

    # Plot
    sns.heatmap(df[missing_df['Column']].isnull(),
                cbar=False,
                cmap='coolwarm',
                yticklabels=False)
    plt.title('Missing Data Heatmap', fontsize=14)
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

    # Print missing percentage for reference
    print("\nMissing Data Percentage by Column:")
    display(missing_df)

In [ ]:
plot_missing_data_heatmap(wildfire_df)

In [ ]:
plot_missing_data_heatmap(aqi_df)

In [ ]:
from geopy.geocoders import Nominatim
import time

def get_location_details(latitude, longitude):
    """
    Get detailed location information from coordinates using reverse geocoding.
    
    Args:
        latitude (float): Latitude coordinate
        longitude (float): Longitude coordinate
    
    Returns:
        dict: Dictionary containing location details or None if not found
    """
    geolocator = Nominatim(user_agent="wildfire_aqi_analysis")
    
    try:
        location = geolocator.reverse(f"{latitude}, {longitude}", exactly_one=True, language='en')
        
        if not location:
            return None
        
        address = location.raw.get('address', {})
        
        # Extract different components
        details = {
            'city': address.get('city') or address.get('town') or address.get('village'),
            'county': address.get('county') or address.get('municipality'),
            'state': address.get('state') or address.get('province'),
            'country': address.get('country'),
            'postcode': address.get('postcode'),
            'full_address': location.address
        }
        
        return details
    
    except Exception as e:
        print(f"Error geocoding {latitude}, {longitude}: {e}")
        return None

# Example usage
latitude = 40.7128  # New York City coordinates
longitude = -74.0060

location_info = get_location_details(latitude, longitude)

if location_info:
    print(f"City: {location_info['city']}")
    print(f"County: {location_info['county']}")
    print(f"State: {location_info['state']}")
    print(f"Country: {location_info['country']}")
    print(f"Full Address: {location_info['full_address']}")
else:
    print("Location not found")

In [ ]:
def supplement_wildfire_missing_data(df, lat_col='LATITUDE', lon_col='LONGITUDE', 
                                   county_col='COUNTY', batch_size=50, delay=1):
    """
    Supplement missing county information in wildfire data using reverse geocoding.
    
    Args:
        df (pd.DataFrame): Wildfire dataframe
        lat_col (str): Name of latitude column
        lon_col (str): Name of longitude column
        county_col (str): Name of county column to supplement
        batch_size (int): Number of records to process before showing progress
        delay (float): Delay between API calls to respect rate limits
    
    Returns:
        pd.DataFrame: DataFrame with supplemented data
    """
    df_copy = df.copy()
    
    # Find rows with missing county data but valid coordinates
    missing_county = df_copy[county_col].isna()
    has_coordinates = df_copy[lat_col].notna() & df_copy[lon_col].notna()
    to_supplement = missing_county & has_coordinates
    
    if not to_supplement.any():
        print("No records need county supplementation")
        return df_copy
    
    print(f"Found {to_supplement.sum()} records with missing county data that have coordinates")
    
    count = 0
    for idx in df_copy[to_supplement].index:
        lat = df_copy.loc[idx, lat_col]
        lon = df_copy.loc[idx, lon_col]
        
        location_info = get_location_details(lat, lon)
        
        if location_info and location_info['county']:
            df_copy.loc[idx, county_col] = location_info['county']
            count += 1
        
        # Progress update and rate limiting
        if (count + 1) % batch_size == 0:
            print(f"Processed {count + 1} records...")
        
        time.sleep(delay)  # Rate limiting
    
    print(f"Successfully supplemented {count} county records")
    return df_copy

def supplement_aqi_missing_data(df, lat_col='Latitude', lon_col='Longitude', 
                               city_col='City', county_col='County', state_col='State',
                               batch_size=50, delay=1):
    """
    Supplement missing location information in AQI data using reverse geocoding.
    
    Args:
        df (pd.DataFrame): AQI dataframe
        lat_col (str): Name of latitude column
        lon_col (str): Name of longitude column
        city_col (str): Name of city column to supplement
        county_col (str): Name of county column to supplement
        state_col (str): Name of state column to supplement
        batch_size (int): Number of records to process before showing progress
        delay (float): Delay between API calls to respect rate limits
    
    Returns:
        pd.DataFrame: DataFrame with supplemented data
    """
    df_copy = df.copy()
    
    # Find rows with missing location data but valid coordinates
    has_coordinates = df_copy[lat_col].notna() & df_copy[lon_col].notna()
    missing_any_location = (df_copy[city_col].isna() | 
                           df_copy[county_col].isna() | 
                           df_copy[state_col].isna())
    to_supplement = missing_any_location & has_coordinates
    
    if not to_supplement.any():
        print("No records need location supplementation")
        return df_copy
    
    print(f"Found {to_supplement.sum()} records with missing location data that have coordinates")
    
    count = 0
    for idx in df_copy[to_supplement].index:
        lat = df_copy.loc[idx, lat_col]
        lon = df_copy.loc[idx, lon_col]
        
        location_info = get_location_details(lat, lon)
        
        if location_info:
            # Only update if the field is currently missing
            if pd.isna(df_copy.loc[idx, city_col]) and location_info['city']:
                df_copy.loc[idx, city_col] = location_info['city']
            
            if pd.isna(df_copy.loc[idx, county_col]) and location_info['county']:
                df_copy.loc[idx, county_col] = location_info['county']
            
            if pd.isna(df_copy.loc[idx, state_col]) and location_info['state']:
                df_copy.loc[idx, state_col] = location_info['state']
            
            count += 1
        
        # Progress update and rate limiting
        if (count + 1) % batch_size == 0:
            print(f"Processed {count + 1} records...")
        
        time.sleep(delay)  # Rate limiting
    
    print(f"Successfully processed {count} location records")
    return df_copy

In [ ]:
# Analyze what columns we have in our datasets for geocoding supplementation
print("=== WILDFIRE DATASET COLUMNS ===")
print("Available columns:")
print(wildfire_df.columns.tolist())
print(f"\nShape: {wildfire_df.shape}")

print("\n=== AQI DATASET COLUMNS ===")
print("Available columns:")
print(aqi_neighbors_df.columns.tolist())
print(f"\nShape: {aqi_neighbors_df.shape}")

# Check for coordinate columns and location columns that could be supplemented
print("\n=== GEOCODING OPPORTUNITIES ===")

# For wildfire data
wildfire_coords = []
wildfire_locations = []
for col in wildfire_df.columns:
    col_lower = col.lower()
    if any(coord in col_lower for coord in ['lat', 'lon', 'coord']):
        wildfire_coords.append(col)
    elif any(loc in col_lower for loc in ['county', 'city', 'state', 'location']):
        wildfire_locations.append(col)

print(f"Wildfire coordinate columns: {wildfire_coords}")
print(f"Wildfire location columns: {wildfire_locations}")

# For AQI data
aqi_coords = []
aqi_locations = []
for col in aqi_neighbors_df.columns:
    col_lower = col.lower()
    if any(coord in col_lower for coord in ['lat', 'lon', 'coord']):
        aqi_coords.append(col)
    elif any(loc in col_lower for loc in ['county', 'city', 'state', 'location']):
        aqi_locations.append(col)

print(f"\nAQI coordinate columns: {aqi_coords}")
print(f"AQI location columns: {aqi_locations}")

In [ ]:
# Check for missing data in location columns before supplementation
print("=== MISSING DATA ANALYSIS FOR GEOCODING ===")

print("\nWildfire Dataset - Missing Location Data:")
wildfire_location_missing = {
    'City': wildfire_df['* City'].isna().sum(),
    'County': wildfire_df['County'].isna().sum(), 
    'State': wildfire_df['State'].isna().sum(),
    'Has_Coordinates': (~wildfire_df['Latitude'].isna() & ~wildfire_df['Longitude'].isna()).sum()
}

for key, value in wildfire_location_missing.items():
    print(f"  {key}: {value:,} missing/available")

print(f"\nRecords with coordinates but missing county: {(wildfire_df['County'].isna() & ~wildfire_df['Latitude'].isna() & ~wildfire_df['Longitude'].isna()).sum():,}")

print("\nAQI Dataset - Missing Location Data:")
aqi_location_missing = {
    'City': aqi_neighbors_df['city_ascii'].isna().sum(),
    'State_Name': aqi_neighbors_df['state_name'].isna().sum(),
    'Has_Coordinates': (~aqi_neighbors_df['lat'].isna() & ~aqi_neighbors_df['lng'].isna()).sum()
}

for key, value in aqi_location_missing.items():
    print(f"  {key}: {value:,} missing/available")

# Note: AQI dataset doesn't seem to have a county column, so we could add one using geocoding!

In [ ]:
# Demonstrate geocoding supplementation on a small sample
print("=== GEOCODING DEMONSTRATION ===")

# Test the geocoding function first
print("Testing geocoding function with sample coordinates...")
sample_lat, sample_lon = 34.0522, -118.2437  # Los Angeles coordinates
test_location = get_location_details(sample_lat, sample_lon)
print(f"Sample location info: {test_location}")

print("\n" + "="*50)
print("WILDFIRE DATA SUPPLEMENTATION DEMO")
print("="*50)

# Find records with missing county data
missing_county_mask = wildfire_df['County'].isna() & ~wildfire_df['Latitude'].isna() & ~wildfire_df['Longitude'].isna()
missing_county_records = wildfire_df[missing_county_mask]

if len(missing_county_records) > 0:
    print(f"Found {len(missing_county_records)} records with missing county but valid coordinates")
    print("\nSample records with missing county:")
    print(missing_county_records[['* City', 'County', 'State', 'Latitude', 'Longitude']].head())
    
    # Demonstrate on just 2-3 records to avoid rate limiting during demo
    print(f"\nDemonstrating county supplementation on first 2 records...")
    demo_sample = missing_county_records.head(2).copy()
    
    for idx, row in demo_sample.iterrows():
        print(f"\nProcessing record {idx}:")
        print(f"  Current: City='{row['* City']}', County='{row['County']}', State='{row['State']}'")
        print(f"  Coordinates: {row['Latitude']}, {row['Longitude']}")
        
        location_info = get_location_details(row['Latitude'], row['Longitude'])
        if location_info:
            print(f"  Geocoded county: '{location_info['county']}'")
            print(f"  Full address: {location_info['full_address']}")
        else:
            print("  Could not geocode this location")
        
        time.sleep(1)  # Rate limiting
else:
    print("No records found with missing county data")

print("\n" + "="*50)
print("AQI DATA ENHANCEMENT DEMO")
print("="*50)

# The AQI dataset doesn't have county information - we could add it!
print("AQI dataset could benefit from adding county information using coordinates")
print("\nSample AQI records:")
aqi_sample = aqi_neighbors_df[['city_ascii', 'state_name', 'lat', 'lng']].head(3)
print(aqi_sample)

print(f"\nDemonstrating county addition for first AQI record...")
first_aqi = aqi_neighbors_df.iloc[0]
print(f"Current: City='{first_aqi['city_ascii']}', State='{first_aqi['state_name']}'")
print(f"Coordinates: {first_aqi['lat']}, {first_aqi['lng']}")

location_info = get_location_details(first_aqi['lat'], first_aqi['lng'])
if location_info:
    print(f"Could add county: '{location_info['county']}'")
    print(f"Full address: {location_info['full_address']}")
else:
    print("Could not geocode this AQI location")

In [ ]:
# INSTRUCTIONS FOR FULL DATASET GEOCODING
print("="*60)
print("FULL DATASET GEOCODING INSTRUCTIONS")
print("="*60)

print("""
To apply geocoding to your full datasets, use the following code:

1. SUPPLEMENT WILDFIRE MISSING COUNTY DATA:
   # This will fill in missing county information for wildfire records
   wildfire_supplemented = supplement_wildfire_missing_data(
       wildfire_df, 
       lat_col='Latitude', 
       lon_col='Longitude', 
       county_col='County',
       batch_size=25,  # Process 25 records at a time
       delay=1.5       # Wait 1.5 seconds between API calls
   )

2. ADD COUNTY COLUMN TO AQI DATA:
   # First, add a county column to the AQI dataframe
   aqi_neighbors_df['County'] = None
   
   # Then supplement with geocoding (this will take a while for full dataset!)
   aqi_with_counties = supplement_aqi_missing_data(
       aqi_neighbors_df,
       lat_col='lat',
       lon_col='lng', 
       city_col='city_ascii',
       county_col='County',  # New column we just added
       state_col='state_name',
       batch_size=25,
       delay=1.5
   )

3. SAVE PROCESSED DATA:
   # Save the supplemented datasets
   wildfire_supplemented.to_csv('../data/processed/california-wildfires/wildfires_with_locations.csv', index=False)
   aqi_with_counties.to_csv('../data/processed/us-air-quality/aqi_with_counties.csv', index=False)

IMPORTANT NOTES:
- Geocoding API calls have rate limits (usually 1 request per second for Nominatim)
- Processing large datasets will take considerable time
- Consider processing in smaller batches and saving intermediate results
- The full AQI dataset has 817k records - this could take 10+ hours to fully geocode
- Consider filtering to specific date ranges or regions first to reduce processing time

RATE LIMITING CONSIDERATIONS:
- Nominatim allows 1 request per second maximum
- For 817k AQI records at 1.5 second intervals = ~340 hours of processing time
- Consider using paid geocoding services for large datasets (Google Maps API, MapBox, etc.)
""")

# Show the current data shapes and potential processing times
wildfire_missing = (wildfire_df['County'].isna() & 
                   ~wildfire_df['Latitude'].isna() & 
                   ~wildfire_df['Longitude'].isna()).sum()

aqi_total = len(aqi_neighbors_df)

print(f"\nCURRENT DATASET STATUS:")
print(f"- Wildfire records needing county supplementation: {wildfire_missing:,}")
print(f"- AQI records that could get county data: {aqi_total:,}")
print(f"\nESTIMATED PROCESSING TIME (at 1.5 sec per record):")
print(f"- Wildfire supplementation: {(wildfire_missing * 1.5 / 60):.1f} minutes")
print(f"- AQI county addition: {(aqi_total * 1.5 / 3600):.1f} hours")

In [13]:
wildfire_supplemented = supplement_wildfire_missing_data(
    wildfire_df, 
    lat_col='Latitude', 
    lon_col='Longitude', 
    county_col='County',
    batch_size=25,  # Process 25 records at a time
    delay=1.5       # Wait 1.5 seconds between API calls
)

Found 30 records with missing county data that have coordinates
Processed 25 records...
Successfully supplemented 30 county records


In [14]:
# Save supplemented wildfire data
output_path = "../data/processed/california-wildfires/wildfires_supplemented.csv"
wildfire_supplemented.to_csv(output_path, index=False)
print(f"Saved supplemented data to: {output_path}")

# Load existing processed data for comparison
existing_path = "../data/processed/california-wildfires/wildfires_with_locations.csv"
try:
    existing_df = pd.read_csv(existing_path)
    print(f"Loaded existing data from: {existing_path}")
    
    # Compare datasets
    print(f"\nCOMPARISON:")
    print(f"Original data shape: {wildfire_df.shape}")
    print(f"Supplemented data shape: {wildfire_supplemented.shape}")
    print(f"Existing processed data shape: {existing_df.shape}")
    
    # Compare column differences
    original_cols = set(wildfire_df.columns)
    supplemented_cols = set(wildfire_supplemented.columns)
    existing_cols = set(existing_df.columns)
    
    print(f"\nCOLUMN COMPARISON:")
    print(f"Original columns: {len(original_cols)}")
    print(f"Supplemented columns: {len(supplemented_cols)}")
    print(f"Existing processed columns: {len(existing_cols)}")
    
    if supplemented_cols != original_cols:
        print(f"New columns in supplemented: {supplemented_cols - original_cols}")
        print(f"Removed columns from supplemented: {original_cols - supplemented_cols}")
    
    if existing_cols != original_cols:
        print(f"New columns in existing: {existing_cols - original_cols}")
        print(f"Removed columns from existing: {original_cols - existing_cols}")
    
    # Compare missing data in County column
    original_missing_county = wildfire_df['County'].isna().sum()
    supplemented_missing_county = wildfire_supplemented['County'].isna().sum()
    existing_missing_county = existing_df['County'].isna().sum() if 'County' in existing_df.columns else "N/A"
    
    print(f"\nCOUNTY DATA COMPARISON:")
    print(f"Original missing counties: {original_missing_county:,}")
    print(f"Supplemented missing counties: {supplemented_missing_county:,}")
    print(f"Existing processed missing counties: {existing_missing_county}")
    print(f"Counties filled by supplementation: {original_missing_county - supplemented_missing_county}")
    
except FileNotFoundError:
    print(f"No existing processed file found at: {existing_path}")
    print("This appears to be the first processed version.")

Saved supplemented data to: ../data/processed/california-wildfires/wildfires_supplemented.csv
Loaded existing data from: ../data/processed/california-wildfires/wildfires_with_locations.csv

COMPARISON:
Original data shape: (100230, 47)
Supplemented data shape: (100230, 47)
Existing processed data shape: (100230, 20)

COLUMN COMPARISON:
Original columns: 47
Supplemented columns: 47
Existing processed columns: 20
New columns in existing: set()
Removed columns from existing: {'Street Suffix (e.g. apt. 23, blding C)', '* Roof Construction', '* Vent Screen', 'Battalion', 'Distance - Residence to Utility/Misc Structure &gt; 120 SQFT', '# of Damaged Outbuildings < 120 SQFT', 'If Affected 1-9% - Where did fire start?', '* City', 'Distance - Propane Tank to Structure', 'Assessed Improved Value (parcel)', '* Street Number', '* Street Name', 'Structure Defense Actions Taken', '* Eaves', '* Window Pane', '* Street Type (e.g. road, drive, lane, etc.)', 'Community', 'Site Address (parcel)', '# of No

In [15]:
# Detailed comparison of the datasets
print("DETAILED COMPARISON:")

# Show sample of counties that were filled by supplementation
original_missing = wildfire_df['County'].isna()
if original_missing.any():
    print("\nSample records where counties were supplemented:")
    sample_indices = wildfire_df[original_missing].index[:5]  # First 5 missing records
    
    for idx in sample_indices:
        print(f"\nRecord {idx}:")
        print(f"  Original County: {wildfire_df.loc[idx, 'County']}")
        print(f"  Supplemented County: {wildfire_supplemented.loc[idx, 'County']}")
        print(f"  City: {wildfire_df.loc[idx, '* City']}")
        print(f"  State: {wildfire_df.loc[idx, 'State']}")
        print(f"  Coordinates: ({wildfire_df.loc[idx, 'Latitude']}, {wildfire_df.loc[idx, 'Longitude']})")

# Compare with existing processed data structure
print(f"\nCOLUMN DIFFERENCES:")
print(f"Existing processed data appears to be a subset with {existing_df.shape[1]} columns vs {wildfire_df.shape[1]} in original")
print(f"Key columns in existing processed data:")
print(existing_df.columns.tolist()[:10], "..." if len(existing_df.columns) > 10 else "")

# Check if existing processed data has same county completeness
if 'County' in existing_df.columns:
    print(f"\nBoth supplemented and existing data have complete county information!")
    print(f"Existing processed data already filled the same missing counties.")
else:
    print(f"\nExisting processed data doesn't have County column.")

# File sizes comparison
import os
new_file_size = os.path.getsize("../data/processed/california-wildfires/wildfires_supplemented.csv")
existing_file_size = os.path.getsize("../data/processed/california-wildfires/wildfires_with_locations.csv")

print(f"\nFILE SIZES:")
print(f"New supplemented file: {new_file_size / 1024 / 1024:.1f} MB")
print(f"Existing processed file: {existing_file_size / 1024 / 1024:.1f} MB")

DETAILED COMPARISON:

Sample records where counties were supplemented:

Record 78435:
  Original County: nan
  Supplemented County: Yuba County
  City: Brownsville 
  State: CA
  Coordinates: (39.4098639835184, -121.306040400213)

Record 78436:
  Original County: nan
  Supplemented County: Yuba County
  City: Brownsville 
  State: CA
  Coordinates: (39.4097826500257, -121.305075083185)

Record 78437:
  Original County: nan
  Supplemented County: Yuba County
  City: Brownsville 
  State: CA
  Coordinates: (39.4098022003337, -121.304769832957)

Record 78438:
  Original County: nan
  Supplemented County: Yuba County
  City: Brownsville 
  State: CA
  Coordinates: (39.4092070163471, -121.304662266888)

Record 78439:
  Original County: nan
  Supplemented County: Yuba County
  City: Brownsville 
  State: CA
  Coordinates: (39.4073752168849, -121.304868282922)

COLUMN DIFFERENCES:
Existing processed data appears to be a subset with 20 columns vs 47 in original
Key columns in existing processe